In [1]:
import importlib
import torch
import utility
from utility import LeNet
from autoattack_dev import AutoAttack

Re-run the following code to re-import the utility importlib.reload(utility)

In [2]:
net = utility.network()
net.cuda()

LeNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)

In [3]:
orignal_sample = utility.samples(adversarial=False)
utility.evaluate(net, orignal_sample, defense=utility.trimmer, device=torch.device("cuda"))

Check accuracy of the model: 0.98


Step 1: set-up BPDA attacks 

In [4]:
inf_BPDA = AutoAttack(net, norm='Linf', eps=0.3, version='standard', know_defense=utility.trimmer)
inf_BPDA.attacks_to_run = ['apgd-ce']
inf_BPDA.apgd.n_restarts = 1
inf_BPDA.apgd.n_iter = 100

setting parameters for standard version


Step 2: apply the APGD

In [5]:
for images, labels in orignal_sample:
    adv_complete = inf_BPDA.run_standard_evaluation(images, labels, bs=100)

using standard version including apgd-ce
initial accuracy: 98.00%
apgd-ce - 1/1 - 98 out of 98 successfully perturbed
robust accuracy after APGD-CE: 0.00% (total time 0.3 s)
max Linf perturbation: 0.30000, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 0.00%


In [6]:
importlib.reload(utility)

<module 'utility' from '/media/disk/h/usr/czw0078@auburn.edu/projects/rethink_minist_robust/utility.py'>

In [7]:
BPDA_sample = utility.samples(x=adv_complete)

In [8]:
utility.evaluate(net, BPDA_sample, defense=utility.trimmer, device=torch.device("cuda"))

Check accuracy of the model: 0.96


In [ ]:
def defense_BPDA(x, defense=utility.trimmer):
    forward_value = defense(x)
    out = x.clone()
    out.date = forward_value.data
    return out

class 

In [9]:
tx = torch.tensor([4,-1.12]).requires_grad_(True)

In [14]:
loss = torch.matmul(tx, tx)
loss.backward()

In [15]:
tx.grad

tensor([ 8.0000, -2.2400])

In [20]:
ty = torch.tensor([4,-1.12]).requires_grad_(True)
tz = torch.clone(ty)
loss2 = torch.matmul(tz[tz>0], tz[tz>0])
loss2.backward()

In [22]:
ty.grad

tensor([8., 0.])